In [1]:
# Importing all required libraries
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from data import create_data,add_tags,make_dataset
from model import SVD_rating_cross

In [2]:
path_to_links = './ml-latest-small/links.csv'
path_to_movies = './ml-latest-small/movies.csv'
path_to_ratings = './ml-latest-small/ratings.csv'
path_to_tags = './ml-latest-small/tags.csv'

data_ratings = pd.read_csv(path_to_ratings)
data_ratings['title'],data_ratings['genres'],data_ratings['imdbId'],data_ratings['tmdbId'] = None, None, None, None
data_links = pd.read_csv(path_to_links)
data_movies = pd.read_csv(path_to_movies)


In [ ]:
data_ratings = create_data(data_ratings, data_movies, data_links)

  0%|          | 0/100836 [00:00<?, ?it/s]

In [4]:
data_tags = pd.read_csv(path_to_tags)
data_ratings['Tags'] = str(list())


In [5]:
data_ratings = add_tags(data_ratings,data_tags)

  0%|          | 0/3683 [00:00<?, ?it/s]

In [6]:
all_columns = list(data_ratings.columns)
dataframe_train,dataframe_test = pd.DataFrame(columns= all_columns),pd.DataFrame(columns=all_columns)

In [7]:
dataframe_train , dataframe_test = make_dataset(dataframe_train,dataframe_test,data_ratings)

dataframe_train.to_csv('train.csv', index=False)
dataframe_test.to_csv('test.csv', index=False)

  0%|          | 0/610 [00:00<?, ?it/s]

In [3]:
from sklearn.metrics.pairwise import pairwise_distances
from scipy.sparse.linalg import svds
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import math
from sklearn.metrics import precision_score
from model import SVD_rating_cross , rate_pred
from evaluation import find_criteria,find_precision_recall,get_10_rec,get_groundtruth

dataframe_train = pd.read_csv('train.csv')
dataframe_test = pd.read_csv('test.csv')

temp_ls = [0, 1, 2]
data_training = dataframe_train.iloc[:, temp_ls ]
data_testing = dataframe_test.iloc[:, temp_ls]

pred = data_testing.copy()
pred['svd_prediction'] = None

training_data = Dataset.load_from_df(data_training, Reader())
full_training_data = training_data.build_full_trainset()
folds = 5
eval_criteria = ['MAE','RMSE']


In [4]:
print('Total Users are ' + str(len(set(data_training.userId))) + ' | Total movies are ' + str(len(set(data_training.movieId))))


Total Users are 610 | Total movies are 8986


In [5]:
results = SVD_rating_cross(training_data,eval_criteria, folds)

Evaluating MAE, RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.6771  0.6748  0.6769  0.6726  0.6855  0.6774  0.0044  
RMSE (testset)    0.8818  0.8787  0.8824  0.8752  0.8875  0.8811  0.0041  
Fit time          0.55    0.77    0.59    0.55    0.58    0.61    0.08    
Test time         0.07    0.09    0.07    0.13    0.07    0.09    0.02    


In [6]:
print('Average RMSE Error:', sum(results['test_rmse'])/folds)
print('Average MAE Error:', sum(results['test_mae'])/folds)

Average RMSE Error: 0.8811230419735507
Average MAE Error: 0.6773741284109766


In [7]:
pred = rate_pred(full_training_data, pred)

  0%|          | 0/20417 [00:00<?, ?it/s]

In [8]:
find_criteria(pred)

MSE :  0.7969706617034824
MAE :  0.6654503599941225


In [9]:
from recommend import new_movie_recommendation

In [10]:
ls = ['UserId', 'Pred_dict']
recommendation_pred = pd.DataFrame(columns=ls)

In [11]:
users = list(set(data_training['userId']))

In [12]:
import warnings
warnings.filterwarnings('ignore')
svd = SVD()
svd.fit(full_training_data)

for i in tqdm(range(len(users))):
    recommendation_pred = new_movie_recommendation(users[i],data_movies, data_training, recommendation_pred,full_training_data,svd)

  0%|          | 0/610 [00:00<?, ?it/s]

In [13]:
pred , pred_ratings = get_10_rec(recommendation_pred)
true_set,true_set_rating = get_groundtruth(dataframe_test)
precision, recall, ndcg = find_precision_recall(dataframe_test,pred,true_set)


  0%|          | 0/610 [00:00<?, ?it/s]

In [14]:
print('The precision of the SVD approach is:', sum(precision)/len(precision))

The precision of the SVD approach is: 0.20262295081967233


In [15]:
print('The recall of the SVD approach is:', sum(recall)/len(recall))

The recall of the SVD approach is: 0.09841754150429229


In [16]:
p = sum(precision)/len(precision)
r = sum(recall)/len(recall)
f1 = 2*p*r/(p+r)

print("F1 Score: ", f1)

F1 Score:  0.13248485290515052


In [17]:
print("NDCG is:",ndcg)

NDCG is: 0.19199999999999692
